In [2]:
import numpy as np
import os
import sys
from six.moves import cPickle
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.utils import np_utils

#NEW--------------------
import sys
from sklearn.metrics import classification_report,confusion_matrix
import csv
from keras.models import load_model
#-----------------------

In [3]:
# Needed as originally code was for theano backend but now using tensor flow
from keras import backend as K
K.set_image_dim_ordering('th')

In [7]:
if (os.getcwd() == '/content'):
    from google.colab import drive
    drive.mount('/content/gdrive')
    FILE_PATH = '/content/gdrive/My Drive/Level-4-Project/ReproducingPreviousWork\1_5s'
    
else:
    FILE_PATH = os.getcwd()

In [10]:
def load_batch(fpath, label_key='labels'):
    f = open(fpath, 'rb')
    if sys.version_info < (3,):
        d = cPickle.load(f)
    else:
        d = cPickle.load(f, encoding="bytes")
        # decode utf8
        for k, v in d.items():
            # added check as otherwise tries to decode a string
            if type(k) is not str:
                del(d[k])
                d[k.decode("utf8")] = v
    f.close()
    data = d["data"]
    labels = d[label_key]

    data = data.reshape(data.shape[0], 1, 75, 75)
    return data, labels


def load_data():
    path = FILE_PATH

    nb_train_samples = 34720

    X_train = np.zeros((nb_train_samples, 1, 75, 75), dtype="uint8")
    y_train = np.zeros((nb_train_samples,), dtype="uint8")

    for i in range(1, 6):
        fpath = os.path.join(path, 'data_batch_' + str(i))
        data, labels = load_batch(fpath)
        X_train[(i-1)*6944:i*6944, :, :, :] = data
        y_train[(i-1)*6944:i*6944] = labels

    fpath = os.path.join(path, 'test_batch')
    X_test, y_test = load_batch(fpath)

    y_train = np.reshape(y_train, (len(y_train), 1))
    y_test = np.reshape(y_test, (len(y_test), 1))

    return (X_train, y_train), (X_test, y_test)


In [11]:
batch_size = 100
nb_classes = 7
nb_epoch = 5 # orignally 100
data_augmentation = False

# input image dimensions
img_rows, img_cols = 75, 75
# the CIFAR10 images are RGB


# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = load_data()
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

X_train shape: (34720, 1, 75, 75)
34720 train samples
6944 test samples


In [13]:
# Updated to use new version of keras
model = Sequential()

model.add(Convolution2D(64, (5, 5), padding='same', input_shape=(1, img_rows, img_cols), activation='relu'))
model.add(Convolution2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(128, (5, 5), padding='same', activation='relu'))
model.add(Convolution2D(128, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes, activation='softmax'))

In [14]:
# let's train the model using SGD + momentum (how original).
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

In [ ]:
if not data_augmentation:
    print('Not using data augmentation.')
    hist = model.fit(X_train, Y_train,
              batch_size=batch_size,
              nb_epoch=nb_epoch,
              validation_data=(X_test, Y_test),
              shuffle=True)


    #NEW--------------------
    y_pred = model.predict_classes(X_test)
    print(y_pred)
    target_names = ['ArmFasterTowards', 'ArmSlowerTowards', 'CirclingArm', 'Clapping', 'PickingUp', 'Sitting', 'Walking']
#     sys.stdout = open('/home/aleksandar/new_sets/1_5s/results/report_1_5s_64_128_.txt', "w")
    print("CLASSIFICATION REPORT:")
    print(classification_report(np.argmax(Y_test,axis=1), y_pred,target_names=target_names))
    print ("\n")
    print("CONFUSION MATRIX:")
    print(confusion_matrix(np.argmax(Y_test,axis=1), y_pred))

#     with open('/home/aleksandar/new_sets/1_5s/results/graphs_1_5s_64_128.csv', "w") as f:
#        w = csv.writer(f)
#        train_loss = hist.history['loss']
#        val_loss = hist.history['val_loss']
#        train_acc = hist.history['acc']
#        val_acc = hist.history['val_acc']
#        helplist = list(zip(train_loss, train_acc, val_loss, val_acc))
#        w.writerow("train_loss, train_acc, val_loss, val_acc")
#        for val in helplist:
#           w.writerow([val])

	
#     model.save('/home/aleksandar/new_sets/1_5s/results/model_1_5s_64_128.h5')
    #-----------------------

else:
    print('Using real-time data augmentation.')

    # this will do preprocessing and realtime data augmentation
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied)
    datagen.fit(X_train)

    # fit the model on the batches generated by datagen.flow()
    hist = model.fit_generator(datagen.flow(X_train, Y_train,
                        batch_size=batch_size),
                        samples_per_epoch=X_train.shape[0],
                        nb_epoch=nb_epoch,
                        validation_data=(X_test, Y_test))








    #NEW--------------------
    y_pred = model.predict_classes(X_test)
    print(y_pred)
    target_names = ['ArmFasterTowards', 'ArmSlowerTowards', 'CirclingArm', 'Clapping', 'PickingUp', 'Sitting', 'Walking']
#     sys.stdout = open('/home/aleksandar/sets/cifar_noninitialised_A/results/report_noninitialised_A_2_4.txt', "w")
    print("CLASSIFICATION REPORT:")
    print(classification_report(np.argmax(Y_test,axis=1), y_pred,target_names=target_names))
    print ("\n")
    print("CONFUSION MATRIX:")
    print(confusion_matrix(np.argmax(Y_test,axis=1), y_pred))





#     with open('/home/aleksandar/sets/cifar_noninitialised_A/results/graphs_noninitialised_A_2_4.csv', "w") as f:
#        w = csv.writer(f)
#        train_loss = hist.history['loss']
#        val_loss = hist.history['val_loss']
#        train_acc = hist.history['acc']
#        val_acc = hist.history['val_acc']
#        helplist = list(zip(train_loss, train_acc, val_loss, val_acc))
#        w.writerow("train_loss, train_acc, val_loss, val_acc")
#        for val in helplist:
#           w.writerow([val])

	
#     model.save('/home/aleksandar/sets/cifar_noninitialised_A/results/model_noninitialised_A_2_4.h5')
    #-----------------------


Not using data augmentation.


C:\Users\macka\Anaconda3\envs\dataScience\lib\site-packages\ipykernel_launcher.py:7: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  import sys


Train on 34720 samples, validate on 6944 samples
Epoch 1/5
34720/34720 [==============================] - 490s 14ms/step - loss: 0.4891 - acc: 0.8085 - val_loss: 0.8505 - val_acc: 0.7971
Epoch 2/5
34720/34720 [==============================] - 476s 14ms/step - loss: 0.0426 - acc: 0.9861 - val_loss: 1.0559 - val_acc: 0.7804
Epoch 3/5
34720/34720 [==============================] - 476s 14ms/step - loss: 0.0212 - acc: 0.9929 - val_loss: 1.2454 - val_acc: 0.7801
Epoch 4/5
24700/34720 [====================>.........] - ETA: 2:09 - loss: 0.0129 - acc: 0.9961